In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Fri_Feb_21_20:23:50_PST_2025
Cuda compilation tools, release 12.8, V12.8.93
Build cuda_12.8.r12.8/compiler.35583870_0


In [2]:
!pip install torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 4.0 MB/s eta 0:00:0000:0100:010m


In [7]:
!pip install "unsloth[cu128-ampere-torch280] @ git+https://github.com/unslothai/unsloth.git"

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-nm6ugpds/unsloth_0fd9c8694fc34bcca82fdf278756df42
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-nm6ugpds/unsloth_0fd9c8694fc34bcca82fdf278756df42
^C
ERROR: Operation cancelled by user


In [5]:
pip install modelscope

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 13.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os;
os.environ['UNSLOTH_USE_MODELSCOPE'] = '1'
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None    #自动发现需要的数据类型
load_in_4bit = True #4位量化，缩小模型容量。但会牺牲精度(资源有限)
# 使用llama 3.1 8B
model,tokenizer = FastLanguageModel.from_pretrained(
    model_name="./MODEL",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.527 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

./MODEL does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


In [2]:
# LoRA进行微调更新模型1%-10%参数
"""
LoRA:LOW-RANK ADAPTATION OF LARGE LANGUAGE MODELS
矩阵A使用高斯初始化,先降维,矩阵B使用全0初始化再升维,维度控制参数是矩阵的秩r,一般为1,6,8,16
"""
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout= 0,
    bias  = "none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2026.1.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
alpaca_prompt = """你是工业轴承故障诊断助手。
你只能基于输入的特征与z-score推理，禁止编造未提供的信息。
输出必须严格包含四段：结论、关键证据、建议动作、不确定性与下一步。

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset, DatasetDict
from pathlib import Path
def load_alpaca_any(root_dir: str, verbose: bool = True) -> DatasetDict:
    """
    通用 Alpaca(jsonl) 加载器：
    - 递归查找所有 *alpaca*.jsonl
    - 自动按 train / val(validation) / test 分类
    - 每个 split 支持多个文件，自动合并
    适配：IMS(1st/2nd/3rd)、CWRU、XJTU 等
    """
    root = Path(root_dir)

    # 只匹配 Alpaca jsonl，避免 baseline.json / metadata.csv 被读进去
    train_files = sorted(set(root.rglob("*train*alpaca*.jsonl")))
    val_files   = sorted(set(root.rglob("*val*alpaca*.jsonl"))) \
                + sorted(set(root.rglob("*valid*alpaca*.jsonl"))) \
                + sorted(set(root.rglob("*validation*alpaca*.jsonl")))
    test_files  = sorted(set(root.rglob("*test*alpaca*.jsonl")))

    # 去重（val_files 可能重复）
    val_files = sorted(set(val_files))

    if verbose:
        print(f"\n[load_alpaca_any] root = {root.resolve()}")
        print(f"  train files: {len(train_files)}")
        for p in train_files[:5]: print("   -", p)
        if len(train_files) > 5: print("   ...")
        print(f"  val files:   {len(val_files)}")
        for p in val_files[:5]: print("   -", p)
        if len(val_files) > 5: print("   ...")
        print(f"  test files:  {len(test_files)}")
        for p in test_files[:5]: print("   -", p)
        if len(test_files) > 5: print("   ...")

    if len(train_files) == 0 and len(val_files) == 0 and len(test_files) == 0:
        raise FileNotFoundError(
            f"在 {root.resolve()} 下没有找到任何 *alpaca*.jsonl 文件。\n"
            f"请确认你的文件名包含 train/val/test + alpaca + .jsonl"
        )

    data_files = {}
    if train_files: data_files["train"] = [str(p) for p in train_files]
    if val_files:   data_files["validation"] = [str(p) for p in val_files]
    if test_files:  data_files["test"] = [str(p) for p in test_files]

    ds = load_dataset("json", data_files=data_files)

    # 保险：确保字段齐全
    need = {"instruction", "input", "output"}
    for split in ds.keys():
        cols = set(ds[split].column_names)
        if not need.issubset(cols):
            raise ValueError(
                f"{split} split 字段不完整！当前列={cols}\n"
                f"你可能把 baseline/metadata 混进来了，或 jsonl 行结构不对。"
            )

    return ds
from datasets import concatenate_datasets
#文件保存在./Data下，请自行调整
ims  = load_alpaca_any(r"./Data/IMS/ims_alpaca_48k_out/")
cwru = load_alpaca_any(r"./Data/CWRU Bearing Data Center/")
xjtu = load_alpaca_any(r"./Data/XJTU-SY_Bearing_Datasets/xjtu_c3_alpaca_48k_out/")

train_ds = concatenate_datasets([ims["train"], cwru["train"], xjtu["train"]]).shuffle(seed=42)
# validation 可能某个数据集没有，就只拼存在的
eval_parts = []
for ds in [ims, cwru, xjtu]:
    if "validation" in ds:
        eval_parts.append(ds["validation"])
eval_ds = concatenate_datasets(eval_parts).shuffle(seed=42) if eval_parts else None
train_ds = train_ds.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=train_ds.column_names)
eval_ds = eval_ds.map(
    formatting_prompts_func,
    batched=True,
     remove_columns=eval_ds.column_names)


[load_alpaca_any] root = /root/autodl-tmp/LLM/Data/IMS/ims_alpaca_48k_out
  train files: 3
   - Data/IMS/ims_alpaca_48k_out/1st_test_train_alpaca.jsonl
   - Data/IMS/ims_alpaca_48k_out/2nd_test_train_alpaca.jsonl
   - Data/IMS/ims_alpaca_48k_out/3rd_test_train_alpaca.jsonl
  val files:   3
   - Data/IMS/ims_alpaca_48k_out/1st_test_val_alpaca.jsonl
   - Data/IMS/ims_alpaca_48k_out/2nd_test_val_alpaca.jsonl
   - Data/IMS/ims_alpaca_48k_out/3rd_test_val_alpaca.jsonl
  test files:  9
   - Data/IMS/ims_alpaca_48k_out/1st_test_test_alpaca.jsonl
   - Data/IMS/ims_alpaca_48k_out/1st_test_train_alpaca.jsonl
   - Data/IMS/ims_alpaca_48k_out/1st_test_val_alpaca.jsonl
   - Data/IMS/ims_alpaca_48k_out/2nd_test_test_alpaca.jsonl
   - Data/IMS/ims_alpaca_48k_out/2nd_test_train_alpaca.jsonl
   ...

[load_alpaca_any] root = /root/autodl-tmp/LLM/Data/CWRU Bearing Data Center
  train files: 1
   - Data/CWRU Bearing Data Center/train_alpaca.jsonl
  val files:   1
   - Data/CWRU Bearing Data Center/val_al

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
#设置训练参数
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset= eval_ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,#指定处理数据时使用的并行进程数，以加快数据预处理速度。
    packing = False, # 可以让短序列的训练速度提高5倍。
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 16,
        warmup_steps = 5,
        max_steps = 60,  # 微调步数
        learning_rate = 2e-4, # 学习率
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        per_device_eval_batch_size=2,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [5]:
#开始训练
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 315,946 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 16 x 1) = 32
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,2.521200
2,2.490100
3,2.507800
4,2.435000
5,2.306500
6,2.039400
7,1.797700
8,1.629400
9,1.378200
10,1.180500


In [6]:
#保存LoRA模型
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")#（2024/06/16修改，官方库变化）

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

In [12]:
pip install --force-reinstall --no-deps unsloth-zoo unsloth

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 6.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.7/405.7 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: unsloth-zoo
    Found existing installation: unsloth_zoo 2026.1.4
    Uninstalling unsloth_zoo-2026.1.4:
      Successfully uninstalled unsloth_zoo-2026.1.4
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2026.1.4
    Uninstalling unsloth-2026.1.4:
      Successfully uninstalled unsloth-2026.1.4
Note: you may need to restart the kernel to use updated packages.


In [13]:
#合并模型并量化成4位gguf保存 （目前colab有空间限制，可能会转换失败）
model.save_pretrained_gguf("model_GGUF", tokenizer, quantization_method = "q4_k_m")

Unsloth: Merging model weights to 16-bit format...
Detected local model directory: /root/autodl-tmp/LLM/MODEL
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [00:39<00:00,  9.81s/it]


Unsloth: Merge process complete. Saved to `/root/autodl-tmp/LLM/model_GGUF`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['MODEL.BF16.gguf']
Unsloth: [2] Converting GGUF bf16 into q4_k_m. This might take 10 minutes...


RuntimeError: Unsloth: GGUF conversion failed: Unsloth: Quantization failed for MODEL.Q4_K_M.gguf
You might have to compile llama.cpp yourself, then run this again.
You do not need to close this Python program. Run the following commands in a new terminal:
You must run this in the same folder as you're saving your model.
git clone --recursive https://github.com/ggerganov/llama.cpp
cd llama.cpp && make clean && make all -j
Once that's done, redo the quantization.
Error: {e}